# Exercises - Pricing a Callable Bond


#### Notation Commands

$$\newcommand{\Black}{\mathcal{B}}
\newcommand{\Blackcall}{\Black_{\mathrm{call}}}
\newcommand{\Blackput}{\Black_{\mathrm{put}}}
\newcommand{\EcondS}{\hat{S}_{\mathrm{conditional}}}
\newcommand{\Efwd}{\mathbb{E}^{T}}
\newcommand{\Ern}{\mathbb{E}^{\mathbb{Q}}}
\newcommand{\Tfwd}{T_{\mathrm{fwd}}}
\newcommand{\Tunder}{T_{\mathrm{bond}}}
\newcommand{\accint}{A}
\newcommand{\carry}{\widetilde{\cpn}}
\newcommand{\cashflow}{C}
\newcommand{\convert}{\phi}
\newcommand{\cpn}{c}
\newcommand{\ctd}{\mathrm{CTD}}
\newcommand{\disc}{Z}
\newcommand{\done}{d_{1}}
\newcommand{\dt}{\Delta t}
\newcommand{\dtwo}{d_{2}}
\newcommand{\flatvol}{\sigma_{\mathrm{flat}}}
\newcommand{\flatvolT}{\sigma_{\mathrm{flat},T}}
\newcommand{\float}{\mathrm{flt}}
\newcommand{\freq}{m}
\newcommand{\futprice}{\mathcal{F}(t,T)}
\newcommand{\futpriceDT}{\mathcal{F}(t+h,T)}
\newcommand{\futpriceT}{\mathcal{F}(T,T)}
\newcommand{\futrate}{\mathscr{f}}
\newcommand{\fwdprice}{F(t,T)}
\newcommand{\fwdpriceDT}{F(t+h,T)}
\newcommand{\fwdpriceT}{F(T,T)}
\newcommand{\fwdrate}{f}
\newcommand{\fwdvol}{\sigma_{\mathrm{fwd}}}
\newcommand{\fwdvolTi}{\sigma_{\mathrm{fwd},T_i}}
\newcommand{\grossbasis}{B}
\newcommand{\hedge}{\Delta}
\newcommand{\ivol}{\sigma_{\mathrm{imp}}}
\newcommand{\logprice}{p}
\newcommand{\logyield}{y}
\newcommand{\mat}{(n)}
\newcommand{\nargcond}{d_{1}}
\newcommand{\nargexer}{d_{2}}
\newcommand{\netbasis}{\tilde{\grossbasis}}
\newcommand{\normcdf}{\mathcal{N}}
\newcommand{\notional}{K}
\newcommand{\pfwd}{P_{\mathrm{fwd}}}
\newcommand{\pnl}{\Pi}
\newcommand{\price}{P}
\newcommand{\probexer}{\hat{\mathcal{P}}_{\mathrm{exercise}}}
\newcommand{\pvstrike}{K^*}
\newcommand{\refrate}{r^{\mathrm{ref}}}
\newcommand{\rrepo}{r^{\mathrm{repo}}}
\newcommand{\spotrate}{r}
\newcommand{\spread}{s}
\newcommand{\strike}{K}
\newcommand{\swap}{\mathrm{sw}}
\newcommand{\swaprate}{\cpn_{\swap}}
\newcommand{\tbond}{\mathrm{fix}}
\newcommand{\ttm}{\tau}
\newcommand{\value}{V}
\newcommand{\vega}{\nu}
\newcommand{\years}{\tau}
\newcommand{\yearsACT}{\tau_{\mathrm{act/360}}}
\newcommand{\yield}{Y}$$


# 1. Black's Formula for Bond Options


In [64]:
# Setup
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq

OPTION_DATA = './data/option_data_bb_SFRZ5.xlsx'
REF_RATES_DATA = './data/ref_rates.xlsx'
DISCOUNT_CURVE_DATA = './data/discount_curve_2025-02-13.xlsx'

## 1.1.

Consider a bond with:
* `T=3`
* face value of `N=100`
* coupons at `annual` frequency
* annualized coupon rate of `cpn=6%`.

Use the bond-pricing formula along with the discount rates in the data file to price this bond.


In [61]:
df = pd.read_excel(DISCOUNT_CURVE_DATA, 'discount curve', parse_dates=['maturity date'])
df.set_index('ttm', inplace=True)

price = 0
coupon = 100*0.06
for i in range(1,4):
    discount = df.loc[i]['discount']
    price += (coupon * discount)
    if(i == 3):
        price += (100 * discount)

print(f"The price of this bond is ${price:0.2f}.")

The price of this bond is $104.99.


## 1.2.

Suppose the bond is callable by the issuer.

* `European` style
* expiration of `Topt=1.5`
* (clean) `strike=100`
* vol of `2.68%`
* forward price of `103.31.`

What is the value of the issuer's call option?


In [ ]:
discount = df.loc[1.5]['discount']
future_price = 103.31
strike = 100
vol = 0.0268
topt=1.5

d_1 = (np.log(future_price/strike) + topt*((np.pow(vol,2)/2))) / (vol*(np.sqrt(topt)))
d_2 = d_1 - (vol*(np.sqrt(topt)))

call_price = discount*(future_price*norm.cdf(d_1) - strike*norm.cdf(d_2))

print(f"The value of the issuer's call option is ${call_price:0.2f}.")

The value of the issuer's call option is $3.37


## 1.3.

What is the price of the callable bond? 

The **callable** bond is the bond issued with an embedded call option (long the issuer.) Thus, it is the value of the vanilla bond minus the value of the call option.


In [63]:
price -= call_price

print(f"The value of the callable bond is ${price:0.2f}.")

The value of the callable bond is $101.61.


## 1.4.

Which assumptions of Black's formula do we prefer to Black-Scholes for this problem?


We prefer Black’s formula because the option is written on a forward bond price. Black assumes the forward price is lognormally distributed under the risk-neutral measure and discounts the expected payoff using the appropriate discount factor. This is more natural in fixed-income settings than Black-Scholes, which models the spot price directly.

## 1.5

Redo 1.2. Suppose the market prices the call option at `3.50`.

Solve for the implied volatility.


In [78]:
discount = df.loc[1.5]['discount']
future_price = 103.31
strike = 100
vol = 0.0268
topt=1.5

def f(vol):
    d_1 = (np.log(future_price/strike) + (topt * np.pow(vol,2) / 2)) / (vol * np.sqrt(topt))
    d_2 = d_1 - (vol * np.sqrt(topt))
    price = discount * (future_price * norm.cdf(d_1) - strike * norm.cdf(d_2))
    return price-3.5


implied_vol = brentq(f,0.0001,1)*100
print(f"The inplied volatility is {implied_vol:0.2f}%")


The inplied volatility is 3.09%
